In [25]:
from pysis.flowsheet import Simulation
import time
import numpy
from scipy.stats import _qmc
import pysis.flowsheet as sim_f

fs = Simulation(path=f'C:\\Users\\LuanNogueira\Downloads\\Ic-Metamodelagem\DWSIMSIMS\\HidrogenaçãoCO2A.hsc')
fs.update_flowsheet()
fs.set_visible(visibility=1)
operations=fs.Operations
materialStreams=fs.MatStreams
distillationColumn=sim_f.DistillationColumn(fs.Operations['T-100'].COMObject)
Pfr=sim_f.PFR(fs.Operations['PFR-100'].COMObject)
fs.Solver.CanSolve=True


def ISBL():
    def CustoReator():
        return 4.00*(61500+32500*Pfr.get_volume()**0.80)
    def CustoTC():
        #Custo de cada trocador baseado na área de troca térmica 
        #Penso que a área de troca térmica não seja um parâmetro bom pra calcular custo
        x=0
        for i in [u for u in operations.keys() if operations[u].classification=='Heat Transfer Equipment']:
            x+=1.0*(28000+54*operations[i].COMObject.HeatTransferArea()**1.20)
        return x
    def CustoTR():
        return 2.5*(170000+1500*(fs.MatStreams['WasteWater'].COMObject.IdealLiquidVolumeFlowValue*1000)**0.90)
    # def CustoFornalha():
    #     return None
    def CustoTrays():
        #Custo de cada "trays" da Torre de destilação * quantidade de trays
        return 52*1.0*(130+440*(3.842)**1.80)
    def CustoComp():
        #Custo de cada compressor somado
        x=0
        for i in [u for u in operations.keys() if operations[u].classification=='Rotating Equipment']:
            x+=2.5*(580000+20000*operations[i].COMObject.EnergyValue**0.60)
        return x
    return CustoReator()+CustoTC()+CustoComp()+CustoTrays()



def CustoUtilidades():
    def CustoResfriamento():
        return fs.MatStreams['UtilidadeFria'].COMObject.IdealLiquidVolumeFlowValue*3600*0.03
    def CustoAquecimento():
        return fs.EnerStreams['CustoAquecimento'].get_power()*1e6*3600*12.1
    def CustoWasteWater():
        return fs.MatStreams['WasteWater'].COMObject.IdealLiquidVolumeFlowValue*3600*1.5
    def CustoEletricidade():
        return fs.EnerStreams['CustoEletricidade'].get_power()*1e3*3600*94.5
    return CustoAquecimento()+CustoResfriamento()+CustoEletricidade()+CustoWasteWater()


def CustoMateriaPrima():
    def CustoCO2():
        return fs.MatStreams['1'].get_massflow()/1000*95.50
    def CustoH2():
        return fs.MatStreams['9'].COMObject.IdealLiquidVolumeFlowValue*3600*0.165
    return CustoCO2()+CustoH2()

def simulated_values() ->list: 
    # Variáveis de interesse: 
    # Vazões de utilidade/MP
    # Custos Equipamentos 
    # Metanol Produzido 
    
   return [ISBL(),CustoUtilidades(),CustoMateriaPrima()]

def set_dof(x:list):
    
    fs.Solver.CanSolve=False

    materialStreams['13'].set_temperature(x[0],'C')

    materialStreams['9'].set_molarflow(x[1],'kgmole/h')
    
    fs.Operations['TEE-100'].SplitsValue=(x[2],1-x[2])

    distillationColumn.set_specifications(spec='Reflux Ratio', value=x[3])
    
    Pfr.set_volume(x[4])

    materialStreams['10'].set_pressure(x[5],'kpa')
    materialStreams['8'].set_pressure(x[5],'kpa')
    materialStreams['REC'].set_pressure(x[5],'kpa')
    fs.Solver.CanSolve = True

def RReciclo():
    materialStreams['REC'].set_molarflow(5.204e4)
    materialStreams['REC'].set_compmolarflow(materialStreams['REC-2'].get_compmolarflow())

def run(x:list):
    start=time.time()
    y=[]
    for _ in x:
        set_dof(_)
        trials=0
        is_converged=False
        is_valid=False
        while(is_converged==False or is_valid==False) and trials<=1:
            set_dof(_)
            if trials>0:
                RReciclo()
            is_valid=all([i>-3000 for i in simulated_values()])

            is_converged=distillationColumn.get_convergence()
            
            trials+=1
        y.append(simulated_values()+[is_valid, is_converged])
    sim_f.Simulation.save(fs)
    end=time.time()
    print(end-start)
    return y
    # fs.close()

sampler=_qmc.LatinHypercube(d=6)
sample=sampler.random(n=2)
_qmc.discrepancy(sample)

l_bounds=[180,4500,0.001,1.25,35,4500]

u_bounds=[240,6500,0.05,1.8,55,5500]

x=list(_qmc.scale(sample,l_bounds, u_bounds))

def OperationPower(operation :str):
    PowerList=[]
    for _ in fs.EnerStreams.keys(): 
        for ops in fs.EnerStreams[_].get_connections().values():
            for l in ops:
                if(fs.Operations[str(l)].classification == operation):
                    PowerList.append(fs.EnerStreams[_].get_power())
    return (PowerList)

def HeaterExchangerDuty():
    y=0
    for u in fs.Operations.keys():
        if (fs.Operations[u].classification == 'Heat Transfer Equipment'):
            y+=operations[u].COMObject.DutyValue()

        

def HeaterExchangerArea():
    y=[]
    for u in fs.Operations.keys():
        if (fs.Operations[u].classification == 'Heat Transfer Equipment'):
            y.append(operations[u].COMObject.HeatTransferArea())
    return y


In [26]:
simulated_values()

[50695247.989162385, 7231087051920561.0, 8427.279978043945]